In [ ]:
import json
import re

input_txt = "/content/data_final.txt"
output_jsonl = "/content/train.jsonl"

with open(input_txt, "r", encoding="utf-8") as f:
    blocks = f.read().split("\n\n---\n\n")

def block_to_messages(block):
    lines = block.strip().splitlines()
    messages = []
    current_role = None
    current_content = []

    for line in lines:
        line = line.rstrip()
        if line.startswith("User:"):

            if current_role and current_content:
                messages.append({
                    "role": current_role,
                    "content": "\n".join(current_content).strip()
                })
                current_content = []
            current_role = "user"
            current_content.append(line[len("User:"):].strip())
        elif line.startswith("Sorachio:"):
            if current_role and current_content:
                messages.append({
                    "role": current_role,
                    "content": "\n".join(current_content).strip()
                })
                current_content = []
            current_role = "assistant"
            current_content.append(line[len("Sorachio:"):].strip())
        else:
            current_content.append(line)

    if current_role and current_content:
        messages.append({
            "role": current_role,
            "content": "\n".join(current_content).strip()
        })

    return messages

with open(output_jsonl, "w", encoding="utf-8") as fout:
    for block in blocks:
        messages = block_to_messages(block)
        if len(messages) >= 2:
            fout.write(json.dumps({"messages": messages}, ensure_ascii=False) + "\n")
